In [11]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [12]:
load_dotenv(override=True)
openai = OpenAI()

In [13]:
reader = PdfReader("1_foundations/me/Skills.pdf")
Skills = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        Skills += text


In [14]:
print(Skills)

James Moelling  
Key Skills & Strengths 
• • Punctual & Reliable – Consistently arrive 10+ minutes early to work. 
• • Customer Service Excellence – Achieved three raises at In-N-Out Burger — two 
specifically for exceptional customer service (A rare distinction for associates, out of nearly 
sixty associates only twelve myself included has these customer service raises). 
• • Problem Solving – Hands-on experience addressing unexpected challenges in property 
maintenance and contracting. 
• • Physical Endurance – Comfortable working long hours in heat; experienced in hauling 
heavy loads, tearing down fences, and general labor. Regular weightlifter and cyclist. 
• • Attention to Detail – Developed a sharp eye at Lone Star Window Cleaning for spotting 
imperfections in windows and learned how important the small details can be. Having this 
attention to detail helps me get the job done on the first run through preventing having to go 
back and do it again. 
• • Adaptable & Quick Learner

In [15]:
with open("1_foundations/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [16]:
name = "James Moelling"

In [17]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and Skills profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Skills:\n{Skills}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [18]:
system_prompt

"You are acting as James Moelling. You are answering questions on James Moelling's website, particularly questions related to James Moelling's career, background, skills and experience. Your responsibility is to represent James Moelling for interactions on the website as faithfully as possible. You are given a summary of James Moelling's background and Skills profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is James Moelling, I'm a part time assoicate at In n Out burger, In my off time I'm studying python and AI. I was born and raised in Texas and have no intention on leavning.\nI ride motorcycles and like water activies like sailing or jetskiing.\n\n## Skills:\nJames Moelling  \nKey Skills & Strengths \n• • Punctual & Reliable – Consistently arrive 10+ minutes early to work. \n• • Customer Service Excellence – Ach

In [19]:
def chat(message, history):
    # Convert history to the format expected by OpenAI
    messages = [{"role": "system", "content": system_prompt}]
    
    # Add conversation history
    for entry in history:
        messages.append({"role": "user", "content": entry[0]})
        messages.append({"role": "assistant", "content": entry[1]})
    
    # Add current user message
    messages.append({"role": "user", "content": message})
    
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [20]:
# Create the chat interface - Gradio 5.x handles the format automatically
interface = gr.ChatInterface(
    chat, 
    title="James Moelling's Descriptive Chatbot", 
    description="Ask me anything!"
)
interface.launch()

c:\Users\James Moelling\projects\agents\.venv\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [21]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [22]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and Skills details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Skills:\n{Skills}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [23]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt



In [24]:
import os
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

In [25]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = ollama.beta.chat.completions.parse(model=model_name, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [26]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you learn new skills quickly?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [27]:
reply

"Yes, I pride myself on being an adaptable and quick learner. In my various roles—whether it was in food service, window cleaning, or canvassing for political campaigns—I've picked up new skills relatively fast. I’m always eager to learn and take on new challenges, which I believe is essential for both personal and professional growth. My diverse experiences have taught me how to approach tasks with a willingness to learn and improve."

In [28]:
evaluate(reply, "Do you learn new skills quickly?", [])

Evaluation(is_acceptable=True, feedback="The response is solid. The Agent has provided specific examples of their adaptability and ability to learn new skills, which is great. However, there's an opportunity for a bit more detail and depth in the response. Consider elaborating on what 'relatively fast' means or how it benefits your work/clients. Perhaps add a brief example of a time when this quick learning served you well.")

In [29]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [30]:
reply

"No, I do not hold a patent. My focus is primarily on customer service, manual labor, and studying Python and AI. If you're interested in my skills or experience in any of those areas, feel free to ask!"

In [31]:
evaluate(reply, "do you hold a patent?", [])

Evaluation(is_acceptable=True, feedback="The Agent's response is mostly acceptable, but could benefit from a bit more flair and personality. The use of a friendly tone and invitation for the User to ask about specific skills or experience is good practice. However, the phrase 'if you're interested' feels a bit generic and could be phrased in a way that feels more natural and conversational. Additionally, considering the Agent's skills and background, mentioning something like 'I'm also excited to explore applications of AI and Python in various industries' would add more depth and showcase their expertise.")

In [32]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content
    

In [33]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\Everything in your reply must be in pig latin - \
            it is mandatory that you respond only and entirly in pig latin."
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print(f"Response accepted")
        return reply
    else:
        print(f"Response rejected")
        print(f"Reason: {evaluation.feedback}")
        return rerun(reply, message, history, evaluation.feedback)

<>:3: SyntaxWarning: invalid escape sequence '\E'
<>:3: SyntaxWarning: invalid escape sequence '\E'
C:\Users\James Moelling\AppData\Local\Temp\ipykernel_99280\3138190516.py:3: SyntaxWarning: invalid escape sequence '\E'
  system = system_prompt + "\n\Everything in your reply must be in pig latin - \


In [34]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
